## The Culture of International Relations - Text Analysis
### <span style='color: green'>SETUP </span> Prepare and Setup Notebook <span style='float: right; color: red'>MANDATORY</span>

In [ ]:
# Setup
%load_ext autoreload
%autoreload 2

import sys
import ipywidgets as widgets

sys.path = list(set(['..', '../3_text_analysis']) - set(sys.path)) + sys.path

import matplotlib.pyplot as plt
import common.utility as utility
import common.treaty_utility as treaty_utility
import common.treaty_state as treaty_repository
import common.config as config
import treaty_corpus
import textacy_corpus_utility as textacy_utility
#from beakerx.object import beakerx
#from beakerx import *
from IPython.display import display, set_matplotlib_formats

logger = utility.getLogger('corpus_text_analysis')

treaty_repository.load_wti_index_with_gui(data_folder=config.DATA_FOLDER)

%matplotlib inline

In [ ]:
# Create "LTS+UNTS GCAs, 1935-1972"
# See https://github.com/humlab/the_culture_of_international_relations/issues/28

import sys
import os
if '..' not in sys.path: sys.path.append('..')

#from common.treaty_state import load_wti_index
#wti_index = load_wti_index('../data/')

import treaty_corpus

source_folder = '../data'
#source_file = 'treaty_text_corpora_en_201908.zip'
source_file = 'Treaty texts (.txt) (finished).zip'
source_path = os.path.join(source_folder, source_file)

treaties = treaty_repository.current_wti_index().treaties
treaties = treaties.loc[treaties.is_cultural_yesno_gen == 'yes']
treaties = treaties.loc[treaties.english == 'en']
treaties = treaties.loc[treaties.signed_year.isin(range(1935, 1973))]

# FIXED: Missing (304113, 304426, 303720, 10707, 304347, 200212, 304172, 303759, 304520, 104350, 200215, 200225, 304400)
# Treaties not found in archive: 10707

corpus_stream = treaty_corpus.TreatyCompressedFileReader(source_path, 'en', list(treaties.index))
corpus = treaty_corpus.TreatyCorpus(corpus_stream)

In [ ]:
# Compile "LTS+UNTS GCAs, 1935-1972" corpus

import glob
import shutil
folder = "LTS+UNTS_GCAs_1935-1972_20200603"

for treaty_id in treaties.index:
    filepattern = os.path.join(source_folder, folder, "{}_en*.txt".format(treaty_id))
    filenames = glob.glob(filepattern)
    if len(filenames) == 1:        
        #print("file found: {}".format(filenames[0]))
        shutil.move(filenames[0], os.path.join(source_folder, folder, "compiled", os.path.split(filenames[0])[1]))
    elif len(filenames) == 0:
        print("missing file: {}".format(treaty_id))
    else:
        print("duplicate: {}".format(treaty_id))


In [ ]:
import zipfile
with zipfile.ZipFile('../data/tCoIR_en_45-72_renamed.txt.zip', 'w', compression=zipfile.ZIP_DEFLATED) as new_zipfile:
    with zipfile.ZipFile('../data/tCoIR_en_45-72.txt.zip') as old_zipfile:
        for filename in old_zipfile.namelist():
            treaty_id = filename.split('_')[0]
            signed_year = treaty_repository.current_wti_index().treaties.loc[treaty_id]['signed_year']
            new_filename = '{}_{}'.format(signed_year, filename)
            with old_zipfile.open(filename) as textfile:
                content = textfile.read()
            new_zipfile.writestr(new_filename, content)